## Matrix factorization techniques for recommender systems (As a Netflix Prize competition) 
### 🔷 Loss function 
- $\min_{x^*, y^*}\sum_{(u,i)\in\kappa}\Big\{\big(r_{ui}-\langle x_u,y_i\rangle\big)^2 + \lambda\big(\|x_u\|^2 + \|y_i\|^2\big)\Big\}$  
- $\kappa = \{(u,i)~|~1\leq u\leq m,~1\leq i\leq n\}$ : the set of the $(u,i)$ pairs for which $r_{ui}$ is known (the training set)  

### 🔷 Two approaches to minimizing above loss function
1. Stochastic Gradient Descent (SGD) 
    - Given ratings for training set, $R = (r_{ui})\in Mat_{m\times n}$ 
    - Want : $\widetilde{R}=XY^T\approx R$, $\widetilde{R}$ predicts $R$.
    - Compute the associated prediction error :  
        - $e_{ui}:= r_{ui} - \langle x_u, y_i\rangle$    
2. Alternating Least Squares (ALS)  
    - $x_u, y_i$ 모두 미지수이므로 위에서 정의한 loss function은 convex가 아님
    - 둘 중 하나를 상수로 고정하면 최소제곱 문제  
    - 각 단계가 수렴할 때까지 위의 loss function은 감소

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

# the number of factors (latent features)
K = 3

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]]) 
num_users, num_items = R.shape
print('Number of users = %d' % num_users, 'Number of items = %d' % num_items, sep='\n')

# X, Y random initialize
np.random.seed(1)
X = np.random.normal(scale=1.0/K, size=(num_users, K))
Y = np.random.normal(scale=1.0/K, size=(num_items, K))
print('User-latent (%d,%d)-Matirx X = ' % X.shape, X, sep='\n')
print('--------------------------------------------')
print('Item-latent (%d,%d)-Matirx Y = ' % Y.shape, Y, sep='\n')

Number of users = 4
Number of items = 5
User-latent (4,3)-Matirx X = 
[[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]]
--------------------------------------------
Item-latent (5,3)-Matirx Y = 
[[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]


----------------------------
### 1. SGD을 이용한 MF 업데이트
- 기울기의 반대 방향으로 $\gamma$ (learning rate)에 비례하는 크기로 매개변수를 수정하여 결과 출력 :  
    - $x_u ~\leftarrow~ x_u + \gamma(e_{ui}y_i - \lambda x_u)$
    - $y_i ~\leftarrow~ y_i + \gamma(e_{ui}x_u - \lambda y_i)$ 

In [2]:
## SGD 를 이용한 MF 업데이트
# 실제 R 행렬과 예측 행렬의 오차를 구하는 함수
def get_rmse(R, X, Y, non_zeros):
    # error = 0
    # latent rating matrix, predict matrix
    latent_R = X @ Y.T

    # 여기서 non_zeros는 아래 함수에서 확인
    x_non_zero_ind = [non_zeros[0] for non_zeros in non_zeros]
    y_non_zero_ind = [non_zeros[1] for non_zeros in non_zeros]

    # 원 행렬 R에서 0 아닌 값들만 추출
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    # 예측 행렬에서 원 행렬 R에서 0이 아닌 위치의 값들만 추출하여 저장
    latent_R_non_zeros = latent_R[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, latent_R_non_zeros)
    rmse = np.sqrt(mse)

    return rmse


def matrix_factorization(R, steps=200, learning_rate=0.01, r_lambda=0.01):
    
    # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
    non_zeros = [ (u, i, R[u, i]) for u in range(num_users)
                  for i in range(num_items) if R[u, i] > 0 ]
    
    # SGD 기법으로 X, Y 매트릭스를 업데이트
    for step in range(steps):
        for u, i, r in non_zeros:
            # 잔차 구함
            eui = r - X[u, :] @ Y[i, :].T

            # Regularization(lambda-term)을 반영한 SGD 업데이터 적용
            X[u, :] = X[u, :] + learning_rate*(eui * Y[i, :] - r_lambda*X[u, :])
            Y[i, :] = Y[i, :] + learning_rate*(eui * X[u, :] - r_lambda*Y[i, :])

        rmse = get_rmse(R, X, Y, non_zeros)
        if step % 10 == 0:
            print("iter step: {0}, rmse: {1:4f}".format(step, rmse))

    return X, Y

X, Y = matrix_factorization(R)

iter step: 0, rmse: 3.238805
iter step: 10, rmse: 2.916635
iter step: 20, rmse: 2.198707
iter step: 30, rmse: 1.304553
iter step: 40, rmse: 0.756513
iter step: 50, rmse: 0.487672
iter step: 60, rmse: 0.356231
iter step: 70, rmse: 0.279523
iter step: 80, rmse: 0.226563
iter step: 90, rmse: 0.187008
iter step: 100, rmse: 0.156434
iter step: 110, rmse: 0.132346
iter step: 120, rmse: 0.113113
iter step: 130, rmse: 0.097591
iter step: 140, rmse: 0.084944
iter step: 150, rmse: 0.074551
iter step: 160, rmse: 0.065941
iter step: 170, rmse: 0.058753
iter step: 180, rmse: 0.052710
iter step: 190, rmse: 0.047599


In [3]:
print('User-Latent Matrix, X = \n', X)
print('-------------------------------------------------------------')
print('Item-Latent Matrix, Y = \n', Y)

User-Latent Matrix, X = 
 [[ 1.05573236  0.39757377 -0.77039218]
 [-0.06142198  0.94465111 -2.56395289]
 [ 2.4813266   0.13694622 -1.12174128]
 [ 0.5401693   1.11424815 -1.13896042]]
-------------------------------------------------------------
Item-Latent Matrix, Y = 
 [[ 1.61888728  1.41169357 -2.19725822]
 [-0.89031381  0.42042905 -1.76736169]
 [ 1.01945627 -0.04623446 -0.42590928]
 [ 1.08128438  0.22631325 -1.09102838]
 [ 1.38928895 -0.23543601 -0.51337457]]


In [4]:
pred_matrix = np.dot(X, Y.T)
print('Latent rating matrix (Predicted rating matrix) =', pred_matrix, sep='\n')
print('---------------------------------------------------')
print('Given Rating Matrix R =', R, sep='\n') 

Latent rating matrix (Predicted rating matrix) =
[[ 3.96311456  0.58878009  1.38600854  2.07204285  1.76861388]
 [ 6.86778919  4.98327571  0.98571887  2.94471781  1.00853046]
 [ 6.67506941 -0.16906059  3.00103235  3.93786401  3.99091101]
 [ 4.9500403   2.00049712  0.98425613  2.07888389  1.07283042]]
---------------------------------------------------
Given Rating Matrix R =
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


--------------------------------------------------

### 2. ALS 알고리즘에 의한 학습

In [5]:
## equation 8 of paper - p. 47 

#### Binary rating matrix = P : bipartite adjacency block
P = np.copy(R)
P = np.where(P > 0, 1, 0)
print('Binary Rating Matrix P =', P, '(User, Item) = (%d, %d)' % (P.shape[0], P.shape[1]), sep='\n')
print('-------------------------------------------------------------')

#### sparse rating matrix : 결측값은 0으로 채워서 sparse matrix로 만듦 
sparse_R = np.where(R > 0, R, 0)
print('sparse matrix R : \n', sparse_R) 
print('-------------------------------------------------------------')

#### increasing rate of confidence in p_ui=1 (i.e., positive preference)
alpha = 40
#### confidence matrix 
C = 1 + (alpha * sparse_R) 
print('confidence matrix C :\n', C)

Binary Rating Matrix P =
[[1 0 0 1 0]
 [0 1 0 1 1]
 [0 0 1 1 1]
 [1 1 1 1 0]]
(User, Item) = (4, 5)
-------------------------------------------------------------
sparse matrix R : 
 [[4. 0. 0. 2. 0.]
 [0. 5. 0. 3. 1.]
 [0. 0. 3. 4. 4.]
 [5. 2. 1. 2. 0.]]
-------------------------------------------------------------
confidence matrix C :
 [[161.   1.   1.  81.   1.]
 [  1. 201.   1. 121.  41.]
 [  1.   1. 121. 161. 161.]
 [201.  81.  41.  81.   1.]]


- https://yeomko.tistory.com/4

In [6]:
# Cu = np.diag(C[0])
# print(Cu.shape, Y.shape, (Y.T @ Y).shape)
# # for i in range(num_items):
# pu = np.array([X[0] @ Y[i].T for i in range(num_items)])
# pi = np.array([Y[0] @ X[u].T for u in range(num_users)])
# print(pu.shape, pi.shape)

In [7]:
# 기본 loss function 
def basic_loss_function(sparse_R, X, Y, r_lambda=40):
    # predict error = (rui - xu.T @ yi)^2 
    pred_error = np.sum(np.square(sparse_R - X @ Y.T))
    # regularization term = lambda * (sum xu-norm + sum yi-norm) : 정규화
    regular = r_lambda * (np.trace(X @ X.T) + np.trace(Y @ Y.T))
    # total loss = pred_error + reg 
    total_loss = pred_error + regular
    return pred_error, regular, total_loss

# optimizer (|users| = 4, |items|=5)
# xu = (Y.T_Y + lambda * I)^{-1} Y.T_ru
# ru = [ru1, ru2, ... ,ru5].T : (5,1)-matrix (or 5-dim vector)
def basic_opt_user(X, Y, sparse_R, num_users, K=3, r_lambda=40):
    for u in range(num_users):
        YT_Y = Y.T @ Y
        lI = 40 * np.identity(K)
        YT_ru = Y.T @ sparse_R[u]
        X[u] = np.linalg.solve(YT_Y + lI, YT_ru)

def basic_opt_item(X, Y, sprase_R, num_items, K=3, r_lambda=40):
    for i in range(num_items):
        XT_X = X.T @ X
        lI = 40 * np.identity(K)
        XT_ri = X.T @ sparse_R[:, i]
        Y[i] = np.linalg.solve(XT_X + lI, XT_ri)

# train 
# usually ALS algorithm repeat train steps for 10~15 times
b_pred_errors, b_regular_list, b_total_losses = [], [], []

for i in range(15):
    if i != 0:
        basic_opt_user(X, Y, sparse_R, num_users)
        basic_opt_item(X, Y, sparse_R, num_items)
    pred = X @ Y.T
    pred_error, regular, total_loss = basic_loss_function(sparse_R, X, Y)

    b_pred_errors.append(pred_error)
    b_regular_list.append(regular)
    b_total_losses.append(total_loss)
    print('------------step %d----------------' % i)
    print('predict error: %f' % pred_error)
    print('regularization: %f' % regular)
    print('total loss: %f' % total_loss)

------------step 0----------------
predict error: 99.292816
regularization: 1560.720443
total loss: 1660.013259
------------step 1----------------
predict error: 128.769788
regularization: 16.044941
total loss: 144.814729
------------step 2----------------
predict error: 129.997547
regularization: 0.028166
total loss: 130.025713
------------step 3----------------
predict error: 129.999995
regularization: 0.000059
total loss: 130.000054
------------step 4----------------
predict error: 130.000000
regularization: 0.000000
total loss: 130.000000
------------step 5----------------
predict error: 130.000000
regularization: 0.000000
total loss: 130.000000
------------step 6----------------
predict error: 130.000000
regularization: 0.000000
total loss: 130.000000
------------step 7----------------
predict error: 130.000000
regularization: 0.000000
total loss: 130.000000
------------step 8----------------
predict error: 130.000000
regularization: 0.000000
total loss: 130.000000
------------ste

In [8]:
basic_predict = X @ Y.T
print('Final prediction for basic als =\n', basic_predict)
print('---------------------------------------------------')
print('Given Rating Matrix R =', R, sep='\n') 

Final prediction for basic als =
 [[2.81561090e-37 2.34468302e-37 1.37004412e-37 3.53142074e-37
  1.66142209e-37]
 [3.68427923e-37 3.06806135e-37 1.79272821e-37 4.62092973e-37
  2.17400171e-37]
 [3.82292064e-37 3.18351414e-37 1.86018954e-37 4.79481780e-37
  2.25581056e-37]
 [4.17922123e-37 3.48022131e-37 2.03356134e-37 5.24170031e-37
  2.46605469e-37]]
---------------------------------------------------
Given Rating Matrix R =
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


---------------------
### 3. weighted ALS

In [9]:
# weighted loss function
def loss_function(C, P, X, Y, r_lambda=40):
    # predict error = (pui - xu.T @ yi)^2 : 0과 1로 나누어 선호/비선호를 예측한 결과의 오차
    pred_error = np.square(P - X @ Y.T)
    # confidence error = cui (pui - xu.T @ yi)^2 : 신뢰수준을 적용한 예측값에 대한 오차
    conf_error = np.sum(C * pred_error)
    # regularization term = lambda * (sum xu-norm + sum yi-norm) : 정규화
    regular = r_lambda * (np.trace(X @ X.T) + np.trace(Y @ Y.T))
    # total loss = conf_error + reg = (conf_level * pred_error) + reg
    total_loss = conf_error + regular
    return np.sum(pred_error), conf_error, regular, total_loss

# optimizer (|users| = 4, |items|=5)
# xu = (Y.T_Cu_Y + lambda * I)^{-1} Y.T_Cu_pu
# pu = [pu1, pu2, ... , pu5].T : (5,1)-matrix (or 5-dim vector)
def opt_user(X, Y, C, P, num_users, K=3, r_lambda=40):
    for u in range(num_users):
        # Y : (5,3)-matrix (items, factors)
        Cu = np.diag(C[u]) # (5,5)-diagonal matrix
        YT_Cu_Y = Y.T @ Cu @ Y # (3,3)-mat
        lI = r_lambda * np.identity(K) 
        # pu = np.array([X[u] @ Y[i].T for i in range(num_items)])
        YT_Cu_pu = Y.T @ Cu @ P[u] #pu
        X[u] = np.linalg.solve(YT_Cu_Y + lI, YT_Cu_pu)

def opt_item(X, Y, C, P, num_items, K=3, r_lambda=40):
    for i in range(num_items):
        # X : (4,3)-matrix (users, factors)
        Ci = np.diag(C[:, i]) # (4,4)-diagonal matrix
        XT_Ci_X = X.T @ Ci @ X # (3,3)-mat
        lI = r_lambda * np.identity(K)
        # pi = np.array([Y[0] @ X[u].T for u in range(num_users)])
        XT_Ci_pi = X.T @ Ci @ P[:, i] #pi
        Y[i] = np.linalg.solve(XT_Ci_X + lI, XT_Ci_pi)

# train 
# usually ALS algorithm repeat train steps for 10~15 times
pred_errors, conf_errors, regular_list, total_losses = [], [], [], []

for i in range(15):
    if i != 0:
        opt_user(X, Y, C, P, num_users)
        opt_item(X, Y, C, P, num_items)
    pred = X @ Y.T
    pred_error, conf_error, regular, total_loss = loss_function(C, P, X, Y)

    pred_errors.append(pred_error)
    conf_errors.append(conf_error)
    regular_list.append(regular)
    total_losses.append(total_loss)
    print('------------step %d----------------' % i)
    print('predict error: %f' % pred_error)
    print('confidence error: %f' % conf_error)
    print('regularization: %f' % regular)
    print('total loss: %f' % total_loss)

------------step 0----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 1----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 2----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 3----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 4----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 5----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 6----------------
predict error: 12.000000
confidence error: 1452.000000
regularization: 0.000000
total loss: 1452.000000
------------step 7---------

In [10]:
predict = X @ Y.T
print('Final prediction =\n', predict)
print('---------------------------------------------------')
print('Given Rating Matrix R =', R, sep='\n') 

Final prediction =
 [[0.83035728 0.78056955 0.68670556 0.82108667 0.71520758]
 [0.90964559 0.85510378 0.75227699 0.89948975 0.78350059]
 [0.96595371 0.9080357  0.79884382 0.95516922 0.8320002 ]
 [0.90500029 0.85073701 0.74843533 0.89489632 0.77949948]]
---------------------------------------------------
Given Rating Matrix R =
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


-----------------------------------------------
### 4. ALS 참고 : `csr_matrix` 이용한 방법

In [11]:
def weighted_ALS(sparse_R, X, Y, r_lambda = 40, alpha = 40, n_iter = 10, rank_size = K):
    import scipy
    from scipy.sparse import csr_matrix

    # Confidence matrix : C = 1+ alpha * r_{ui}
    conf = 1 + (alpha * sparse_R) # sparse 행렬 형태를 유지하기 위해서 1을 나중에 더함

    # X, Y 초기화
    X = csr_matrix(X)
    Y = csr_matrix(Y)
    X_I = scipy.sparse.eye(num_users)
    Y_I = scipy.sparse.eye(num_items)
    
    # 정규화 term: 𝝀I -> (3,3)-identity matrix * lambda
    lambda_I = r_lambda * scipy.sparse.eye(rank_size)
    
    # 반복 시작
    for i in range(n_iter):
        xTx = X.T @ X
        yTy = Y.T @ Y
        
        # Y(item)를 고정해놓고 X(user)에 대해 반복
        # Xu = (yTy + yT(Cu-I)Y + 𝝀I)^{-1} yTCuPu
        for u in range(num_users):
            conf_samp = conf[u,:] # Cu
            pref = conf_samp.copy()
            pref[pref!=0] = 1
            # Cu-I: 위에서 conf에 1을 더하지 않았으니까 I를 빼지 않음 
            CuI = scipy.sparse.diags(conf_samp, 0)
            # yT(Cu-I)Y
            yTCuIY = Y.T @ CuI @ Y
            # yTCuPu
            yTCupu = Y.T @ CuI @ pref.T
            # solve the equation 
            X[u] = scipy.sparse.linalg.spsolve(yTy + yTCuIY + lambda_I, yTCupu)
        
        # X를 고정해놓고 Y에 대해 반복
        # Yi = (xTx + xT(Cu-I)X + 𝝀I)^{-1} xTCiPi
        for i in range(num_items):
            conf_samp = conf[:,i].T#.toarray()
            pref = conf_samp.copy()
            pref[pref!=0] = 1
            
            #Ci-I
            CiI = scipy.sparse.diags (conf_samp, 0)
            # xT(Ci-I)X
            xTCiIX = X.T @ CiI @ X
            # xTCiPi
            xTCiPi = X.T @ CiI @ pref.T
            
            Y[i] = scipy.sparse.linalg.spsolve(xTx + xTCiIX + lambda_I, xTCiPi)
            
        return X, Y.T

In [12]:
X, Y_T = weighted_ALS(sparse_R, X, Y)
pred_als = X @ Y_T
print('Final prediction of ALS-model =\n', pred_als.toarray())
print('---------------------------------------------------')
print('Given Rating Matrix R =', R, sep='\n') 

Final prediction of ALS-model =
 [[0.83090211 0.78122541 0.68792949 0.81800898 0.71609303]
 [0.90930751 0.85494323 0.75284374 0.89519776 0.78366485]
 [0.96535566 0.90764045 0.79924773 0.95037621 0.83196859]
 [0.90408568 0.8500336  0.74852043 0.89005696 0.77916454]]
---------------------------------------------------
Given Rating Matrix R =
[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


-------------------------------
### 5. `implicit` 라이브러리 이용한 방법

In [13]:
# using `implicit` library for ALS
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares as ALS

alpha = 40
# user_vectors = xu, item_vectors = yi
inp = csr_matrix(alpha * sparse_R)
als_model = ALS(factors=3, regularization=40, iterations=15)
pred = als_model.fit(inp)
# pred 이용해서 rmse 계산 등등

  0%|          | 0/15 [00:00<?, ?it/s]